In [1]:
import torch
import numpy as np
from Networks.mesh2mesh import *
from Networks.rgbd2mesh import RGBD2Mesh
import trimesh
from Networks.obj2pointcloud import *
from Networks.pointclouddataset import *
from chamferdist import ChamferDistance
from Networks.foldingnet import FoldingNetDec
from Networks import train_mesh2mesh

In [2]:
##Overfitting the stage1 autoencoder

config = {
    'experiment_name': 'pcd_stage1_overfit',
    'device': 'cuda:0',                   # change this to cpu if you do not have a GPU
    'is_overfit': True,                   # True since we're doing overfitting
    'batch_size': 1,
    'resume_ckpt_en': None,
    'resume_ckpt_de': None,
    'learning_rate': 0.0001,
    'max_epochs': 300,
    'print_every_n': 100,
    'validate_every_n': 100,
}

In [3]:



train_mesh2mesh.main(config)


Using device: cuda:0
[001/00019] train_loss: 0.0146281156
[001/00019] val_loss: 0.0055451854
[002/00039] train_loss: 0.0050596787
[002/00039] val_loss: 0.0056102402
[003/00059] train_loss: 0.0057409305
[003/00059] val_loss: 0.0044969921
[004/00079] train_loss: 0.0043864911
[004/00079] val_loss: 0.0064692358
[006/00019] train_loss: 0.0042367961
[006/00019] val_loss: 0.0039091529
[007/00039] train_loss: 0.0038551498


In [ ]:
torch.cuda.empty_cache()

In [2]:
#training stage 1
config = {
    'experiment_name': 'pcd_stage1',
    'device': 'cuda:0',                   # change this to cpu if you do not have a GPU
    'is_overfit': False,                   # True since we're doing overfitting
    'batch_size': 1,
    #'resume_ckpt_en': '/home/wuha/Hanfeng/TUM/ML3D/ML3Dfinal_2021WS/models/runs/pcd_stage1/encoder_best.pth',
    #'resume_ckpt_de': '/home/wuha/Hanfeng/TUM/ML3D/ML3Dfinal_2021WS/models/runs/pcd_stage1/decoder_best.pth',
    'resume_ckpt_en': None,
    'resume_ckpt_de': None,
    'learning_rate': 0.0001,
    'max_epochs': 300,
    'print_every_n': 100,
    'validate_every_n': 100,
}
train_mesh2mesh.main(config)

Using device: cuda:0
[000/00099] train_loss: 0.0155502473
[000/00099] val_loss: 0.0077714482
[000/00199] train_loss: 0.0060733926
[000/00199] val_loss: 0.0063305791
[000/00299] train_loss: 0.0058198923
[000/00299] val_loss: 0.0059201606
[000/00399] train_loss: 0.0067399081
[000/00399] val_loss: 0.0053625638
[000/00499] train_loss: 0.0057761494
[000/00499] val_loss: 0.0048632259
[000/00599] train_loss: 0.0044168651
[000/00599] val_loss: 0.0050596017
[000/00699] train_loss: 0.0046760304
[000/00699] val_loss: 0.0055093458
[000/00799] train_loss: 0.0045205077
[000/00799] val_loss: 0.0043681399
[000/00899] train_loss: 0.0036720204
[000/00899] val_loss: 0.0041399738
[000/00999] train_loss: 0.0038548924
[000/00999] val_loss: 0.0032809104
[000/01099] train_loss: 0.0040242829
[000/01099] val_loss: 0.0030591630
[000/01199] train_loss: 0.0032909455
[000/01199] val_loss: 0.0026313188
[000/01299] train_loss: 0.0032122224
[000/01299] val_loss: 0.0022898624
[000/01399] train_loss: 0.0039153793
[000/0

In [3]:
torch.cuda.empty_cache()

In [12]:
#stage 1 eval

encoder = PointNetEncoder(numpoints=4000)
encoder.load_state_dict(torch.load(f'./models/runs/pcd_stage1/encoder_best.pth'))
encoder.eval()
decoder = PointNetDecoder(numpoints=4000)
decoder.load_state_dict(torch.load(f'./models/runs/pcd_stage1/decoder_best.pth'))
decoder.eval()
val_dataset = pcd_stage1(split='val')
val_dataloader = torch.utils.data.DataLoader(
        val_dataset,     # Datasets return data one sample at a time; Dataloaders use them and aggregate samples into batches
        batch_size=1,   # The size of batches is defined here
        shuffle=False,   # During validation, shuffling is not necessary anymore
        num_workers=4,   # Data is usually loaded in parallel by num_workers
        pin_memory=True  # This is an implementation detail to speed up data uploading to the GPU
    )  
x=1     
for batch in val_dataloader:
    input = batch.numpy()
    batch = batch.permute(0,2,1)
    recon = decoder(encoder(batch))
    out = recon.permute(0,2,1).detach().numpy()


    
    export_pointcloud_to_obj(f'./stage_1/input_{x}.obj', input[0])
    export_pointcloud_to_obj(f'./stage_1/output_{x}.obj', out[0])


    x +=1
    if x==10: break    

In [4]:
### stage 2 training

config = {
    'experiment_name': 'pcd_stage2',
    'device': 'cuda:0',                   # change this to cpu if you do not have a GPU
    'is_overfit': False,                   # True since we're doing overfitting
    'batch_size': 1,
    #'resume_ckpt_en': '/home/wuha/Hanfeng/TUM/ML3D/ML3Dfinal_2021WS/models/runs/pcd_stage2/encoder_best.pth',
    'resume_ckpt_en': None,
    'encoder_pretrained': '/home/wuha/Hanfeng/TUM/ML3D/ML3Dfinal_2021WS/models/runs/pcd_stage1/encoder_best.pth', 
    'learning_rate': 0.0001,
    'max_epochs': 100,
    'print_every_n': 100,
    'validate_every_n': 1000,
}


In [5]:
from Networks import train_mesh2mesh_stage2


train_mesh2mesh_stage2.main(config)

Using device: cuda:0
[000/00099] train_loss: 0.0011968362
[000/00199] train_loss: 0.0003722542
[000/00299] train_loss: 0.0003123520
[000/00399] train_loss: 0.0003722059
[000/00499] train_loss: 0.0003422368
[000/00599] train_loss: 0.0003199830
[000/00699] train_loss: 0.0003594039
[000/00799] train_loss: 0.0003167391
[000/00899] train_loss: 0.0003410597
[000/00999] train_loss: 0.0003107652
[000/00999] val_loss: 0.0002859108
[000/01099] train_loss: 0.0002776613
[000/01199] train_loss: 0.0002684661
[000/01299] train_loss: 0.0002732890
[000/01399] train_loss: 0.0002782895
[000/01499] train_loss: 0.0002640706
[000/01599] train_loss: 0.0002409278
[000/01699] train_loss: 0.0002408925
[000/01799] train_loss: 0.0002743550
[000/01899] train_loss: 0.0002736160
[000/01999] train_loss: 0.0002669666
[000/01999] val_loss: 0.0002511622
[000/02099] train_loss: 0.0002340237
[000/02199] train_loss: 0.0002668038
[000/02299] train_loss: 0.0002638325
[000/02399] train_loss: 0.0002386169
[000/02499] train_los

KeyboardInterrupt: 

In [6]:
#stage 2 eval

encoder = PointNetEncoder(numpoints=1000)
encoder.load_state_dict(torch.load(f'./models/runs/pcd_stage2/encoder_stage2_best.pth'))
encoder.eval()
decoder = PointNetDecoder(numpoints=4000)
decoder.load_state_dict(torch.load(f'./models/runs/pcd_stage1/decoder_best.pth'))
decoder.eval()
val_dataset = pcd_stage2(split='val')
val_dataloader = torch.utils.data.DataLoader(
        val_dataset,     # Datasets return data one sample at a time; Dataloaders use them and aggregate samples into batches
        batch_size=1,   # The size of batches is defined here
        shuffle=False,   # During validation, shuffling is not necessary anymore
        num_workers=4,   # Data is usually loaded in parallel by num_workers
        pin_memory=True  # This is an implementation detail to speed up data uploading to the GPU
    )  
x=1     
for batch in val_dataloader:
    gt = batch['full'].numpy()
    input = batch['partial'].permute(0,2,1)
    print(gt.shape)
    recon = decoder(encoder(input))
    out = recon.permute(0,2,1).detach().numpy()




    print(out.shape)

    
    export_pointcloud_to_obj(f'./stage_2/full_{x}.obj', gt[0])
    export_pointcloud_to_obj(f'./stage_2/recon_{x}.obj', out[0])
    export_pointcloud_to_obj(f'./stage_2/input_{x}.obj', batch['partial'][0])
    



    x +=1
    if x==10: break 

(1, 4000, 3)
(1, 4000, 3)
(1, 4000, 3)
(1, 4000, 3)
(1, 4000, 3)
(1, 4000, 3)
(1, 4000, 3)
(1, 4000, 3)
(1, 4000, 3)
(1, 4000, 3)
(1, 4000, 3)
(1, 4000, 3)
(1, 4000, 3)
(1, 4000, 3)
(1, 4000, 3)
(1, 4000, 3)
(1, 4000, 3)
(1, 4000, 3)


In [23]:
# retrieval eval:
encoder = PointNetEncoder(numpoints=1000)
encoder.load_state_dict(torch.load(f'./models/runs/pcd_stage2/encoder_stage2_best.pth'))
encoder.cuda()
encoder.eval()
encoder_pretrained = PointNetEncoder(numpoints=4000)
encoder_pretrained.load_state_dict(torch.load(f'./models/runs/pcd_stage1/encoder_best.pth'))
encoder_pretrained.cuda()
encoder_pretrained.eval()

correct = 0
val_dataset = retrieval_dataset('train')#choose 'val' if you want to test on the val set
val_dataloader = torch.utils.data.DataLoader(
        val_dataset,     # Datasets return data one sample at a time; Dataloaders use them and aggregate samples into batches
        batch_size=1,   # The size of batches is defined here
        shuffle=False,   # During validation, shuffling is not necessary anymore
        num_workers=4,   # Data is usually loaded in parallel by num_workers
        pin_memory=True  # This is an implementation detail to speed up data uploading to the GPU
    )
total = len(val_dataset)    






In [10]:
#### load the latent vectors of all 2000 ground truth pcds in to a tensor
latent_vectors=[]
with torch.no_grad():
    for gt in val_dataset.get_gts():
        latent_vectors.append(encoder_pretrained(gt['tensor'][None,:,:].permute(0,2,1).cuda()))
latent_vectors = torch.cat(latent_vectors, dim=0)
print(latent_vectors.shape)   



torch.Size([2000, 1024])
tensor(720, device='cuda:0')
0
0.0


In [24]:
#for each partial pcd from the data set find the minimum latent space distance within all 2000 full pcds,
#if it retrieves the exact obj the partial pcd is from, then it's a hit
with torch.no_grad():
    for batch in val_dataloader:
        input = batch['partial'].permute(0,2,1).cuda()
        latent = encoder(input)
        latent = latent.view(-1)
        norm = torch.norm(latent_vectors-latent, dim=1)
        index = torch.argmin(norm)
        if val_dataset.get_gts()[index]['path'] == batch['full'][0]:
            correct +=1

print(correct)
print(correct/len(val_dataset))


17751
0.9342631578947368


In [22]:
print(correct)
print(correct/len(val_dataset))

908
0.908
